In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss

import os
import datetime
import itertools

# ログ出力用の関数
def printTime(tag):
    print(tag + ':', datetime.datetime.now())

printTime('Inputファイルを表示')
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Inputファイルを表示: 2020-02-14 06:22:45.260001
/kaggle/input/test.csv
/kaggle/input/data_description.txt
/kaggle/input/train.csv
/kaggle/input/.ipynb_checkpoints/train-checkpoint.csv
/kaggle/input/.ipynb_checkpoints/test-checkpoint.csv
/kaggle/input/.ipynb_checkpoints/data_description-checkpoint.txt


#### 学習データの読み込み

In [40]:
train = pd.read_csv('/kaggle/input/train.csv')
test = pd.read_csv('/kaggle/input/test.csv')

# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['SalePrice'], axis=1)
train_y = train['SalePrice']

# テストデータは特徴量のみなので、そのままでよい
test_x = test.copy()

 #### 特徴量作成

In [41]:
# 解析対象とする数値列を設定
targets_num = ['1stFlrSF', 
               '2ndFlrSF'
               'BsmtFinSF1', 
               'Fireplaces', 
               'FullBath', 
               'GarageArea', 
               'GarageCars', 
               'GarageYrBlt', 
               'GrLivArea', 
               'MasVnrArea', 
               'OverallQual', 
               'OverallCond',
               'TotalBsmtSF', 
               'TotRmsAbvGrd', 
               'YearBuilt', 
               'YearRemodAdd']

# 解析対象とするカテゴリ列を設定
targets_category = [
                    'MSSubClass',
                    'MSZoning',
                    'Street',
                    'Alley',
                    'LotShape',
                    'LandContour',
                    'Utilities',
                    'LotConfig',
                    'LandSlope',
                    'Neighborhood',
                    'Condition1',
                    'Condition2',
                    'BldgType',
                    'HouseStyle',
                    'RoofStyle',
                    'RoofMatl',
                    'Exterior1st',
                    'Exterior2nd',
                    'MasVnrType',
                    'ExterQual',
                    'ExterCond',
                    'Foundation',
                    'BsmtQual',
                    'BsmtCond',
                    'BsmtExposure',
                    'BsmtFinType1',
                    'BsmtFinType2',
                    'Heating',
                    'HeatingQC',
                    'CentralAir',
                    'Electrical',
                    'KitchenQual',
                    'Functional',
                    'FireplaceQu',
                    'GarageType',
                    'GarageFinish',
                    'GarageQual',
                    'GarageCond',
                    'PavedDrive',
                    'PoolQC',
                    'Fence',
                    'MiscFeature',
                    'SaleType',
                    'SaleCondition']

# 解析対象とする数値列とカテゴリ列の列名を連結
targets = targets_num
targets.extend(targets_category)

# 解析対象列のみを抽出
train_x = train_x.reindex(columns=targets)
test_x = test_x.reindex(columns=targets)

# Dataframeの列の型がobjectのままだとLabelEncoderでエラーするため、strに変換
# ※変換処理後にinfo()で型を確認しても、objectと表示されるが内部的にはstrに変換されている。
train_x[targets_category] = train_x[targets_category].astype(str)
test_x[targets_category] = test_x[targets_category].astype(str)

# ラベルエンコーディング用に学習データとテストデータをunion
lavel_x = train_x.append(test_x, ignore_index=True)

# カテゴリ変数をlabel encodingする
for c in targets_category:
    # 学習データとテストデータに基づいてラベルを設定
    le = LabelEncoder()
    le.fit(lavel_x[c])
    train_x[c] = le.transform(train_x[c])
    test_x[c] = le.transform(test_x[c])

    
# 数値変数のnanを0に変換する
for c in targets_num:
    train_x[c] = train_x[c].fillna(0)
    test_x[c] = test_x[c].fillna(0)

    

#### モデル作成とバリデーション
XGBoostを使用してモデルを作成します。

In [46]:
printTime('モデルの作成開始')

# 学習データを学習データとバリデーションデータに分ける
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]


# tensorflowの警告抑制
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test_x = scaler.transform(test_x)

# pandas.core.series.Series型のままだと学習時にエラーするので、ndarrayに変換
tr_y = np.asarray(tr_y)
va_y = np.asarray(va_y)

#ニューラルネットワークモデルの生成
#モデルの構成は
#入力(13次元)-1000次元-800次元-100次元-予測値(1次元)
#Sequential()というのは入力から枝分かれせずに書けるモデルのクラス
#Dense()は全結合層
#activation は活性化関数，ここではReLUを用いている

model = Sequential()
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(800, activation = 'relu'))
model.add(Dense(100, activation =  'relu'))
model.add(Dense(1))

# モデルをコンパイル
# コンパイルすることで逆伝搬などが準備される
# 最適化関数として Adam を指定
# 誤差関数には平均2乗誤差を指定
 

model.compile(Adam(lr=1e-3), loss="mean_squared_error")


#トレーニングデータで学習し，テストデータで評価

# batch_size: ミニバッチのデータ数
# epochs: 全部のデータについて処理する回数 1epoch = 1周
# verbose: 表示の形式，0だと何も表示されない
# validation_data: 評価用データ（ここではハイパーパラメータの調整をしないので
#   テストデータをそのまま用いている）

history = model.fit(tr_x, 
                    tr_y, 
                    batch_size=128, 
                    epochs=150, 
                    verbose=1, 
                    validation_data=(va_x, va_y))

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)

# バリデーションデータでのスコア(真の値の対数と予測値の対数の二乗平均平方根誤差 (RMSE))を計算する
rmse = np.sqrt(mean_squared_error(np.log(va_y), np.log(va_pred)))

print(f'RMSE: {rmse:.4f}')

# 予測
pred = model.predict(test_x)

pred = pd.DataFrame(pred)[0]

# 提出用ファイルの作成
submission = pd.DataFrame({'Id': test['Id'], 'SalePrice': pred})
submission.to_csv('/kaggle/output/submission_first.csv', index=False)

printTime('モデルの作成終了')

モデルの作成開始: 2020-02-14 06:28:02.883738
Train on 1095 samples, validate on 365 samples
Epoch 1/150
1095/1095 [==============================] - 1s 494us/sample - loss: 39288069558.5900 - val_loss: 38262174717.1945
Epoch 2/150
1095/1095 [==============================] - 0s 97us/sample - loss: 39251136242.6740 - val_loss: 38187073266.6740
Epoch 3/150
1095/1095 [==============================] - 0s 104us/sample - loss: 39112151447.7297 - val_loss: 37953309463.1452
Epoch 4/150
1095/1095 [==============================] - 0s 95us/sample - loss: 38722610573.4429 - val_loss: 37357951419.2658
Epoch 5/150
1095/1095 [==============================] - 0s 114us/sample - loss: 37774704722.2941 - val_loss: 36045719041.4027
Epoch 6/150
1095/1095 [==============================] - 0s 105us/sample - loss: 35801056384.1169 - val_loss: 33529349004.9753
Epoch 7/150
1095/1095 [==============================] - 0s 113us/sample - loss: 32273044058.2429 - val_loss: 29266159318.6192
Epoch 8/150
1095/1095 [======

#### モデルチューニング

In [43]:
printTime('モデルチューニング開始')

"""
# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth': [4, 5, 6],
    'min_child_weight': [0.3, 0.4, 0.5]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# ループカウント変数
loopcount = 0

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:

    loopcount += 1
    
    score_folds = []
    # クロスバリデーションを行う
    # 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        # モデルの学習を行う
        model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)

        # バリデーションデータでのスコアを計算し、保存する
        va_pred = model.predict(va_x)
        rmse = np.sqrt(mean_squared_error(np.log(va_y), np.log(va_pred)))
        score_folds.append(rmse)

    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

    printTime(f'max_depth: {max_depth}, '\
              f'min_child_weight: {min_child_weight}, '\
              f'score: {score_mean:.5f}')
    
# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

"""

printTime('モデルチューニング終了')

モデルチューニング開始: 2020-02-14 06:23:05.290351
モデルチューニング終了: 2020-02-14 06:23:05.291321
